## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import glob
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
from sklearn.model_selection import KFold
from sklearn.ensemble  import AdaBoostRegressor
from sklearn.tree  import DecisionTreeRegressor
from sklearn.metrics import explained_variance_score
import pickle
import os

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.
    Review the README in ./datasets/troika/ to understand the organization of the .mat files.
    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl, ref_fl, fs, window_length_s, window_shift_s):
    """
    Loads and extracts signals from a troika data file.
    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])
    Args:
        data_fl: (str) filepath to a troika .mat file.
    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    
    data = sp.io.loadmat(data_fl)['sig']
    sig = data[2:]
    print(sig.shape)
    
    ref = scipy.io.loadmat(ref_fl)["BPM0"]
    print(ref.shape)
    ref = np.array([x[0] for x in ref])
    labels, features, signals = [], [], []
    
    for i in range(0, len(ref)):
        shift = fs * window_shift_s
        signal_length = fs * window_length_s
        start = i*shift
        end = start + signal_length
        
        ppg = sig[0, start:end]            
        accx = sig[1, start:end]
        accy = sig[2, start:end]
        accz = sig[3, start:end]
        
        feature, ppg, accx, accy, accz = Featurize(ppg, accx, accy, accz, fs)
        signals.append([ppg, accx, accy, accz])
        labels.append(ref[i])
        features.append(feature)
    
    return (np.array(labels), np.array(features), signals)

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.
    Computes the MAE at 90% availability. 
    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.
    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)
    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]
    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.
    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.
    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    fs = 125
    window_length_s = 8
    window_shift_s = 2    
    
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl, fs, window_length_s, window_shift_s)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def bandpass_filter(signal, fs):
    """
    It suppresses frequencies outside a band.
    Args: 
        signal: numpy array to filter
        fs: sampling frequency (Hz)
        low_f: low frequency cutoff (for example: 40/60.0)
        high_f: high frequency cutoff (for example: 240/60.0)
    Returns:
        filtered signal
    """ 
    b, a = scipy.signal.butter(3, (40/60.0, 240/60.0), btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

def Featurize(ppg, accx, accy, accz, fs):
    """
    Feature extraction from the PPG and IMU (accelerometer) signals.
    Args: (numpy arrays)
      ppg: PPG signal data
      accx: accelerometer x-channel
      accy: accelerometer y-channel
      accz: accelerometer z-channel
      fs: sampling frequency (Hz)
    Returns:
        n-tuple of features, ppg, accx, accy, accz
    """
    # Bandpass filter the signals
    ppg = bandpass_filter(ppg, fs)
    accx = bandpass_filter(accx, fs)
    accy = bandpass_filter(accy, fs)
    accz = bandpass_filter(accz, fs)
    
    mn_p = np.mean(ppg)
    mn_x = np.mean(accx)
    mn_y = np.mean(accy)
    mn_z = np.mean(accz)
    
  # Fourier Transform 
    fft_len = max(len(accx), 2046)   #array of frequency bins
    fft_freqs = np.fft.rfftfreq(fft_len, 1 / fs)
    freqs_bw = lambda low, high: (fft_freqs >= low) & (fft_freqs <= high) #select frequency bins
  # Accelerometer magnitude
    accm = np.sqrt(np.sum(np.square(np.vstack((accx, accy, accz))), axis=0))
  # FFT of the accelerometer signal
    fft_x = np.fft.rfft(accx, fft_len)
    fft_y = np.fft.rfft(accy, fft_len)
    fft_z = np.fft.rfft(accz, fft_len)
    fft_m = np.fft.rfft(accm, fft_len)
  # Energy spectrum
    spec_energy_x = np.square(np.abs(fft_x))
    spec_energy_y = np.square(np.abs(fft_y))
    spec_energy_z = np.square(np.abs(fft_z))
    spec_energy_m = np.square(np.abs(fft_m))
  # Maximum frequency between 0.25-12 Hz
    dom_x = fft_freqs[np.argmax(fft_x[freqs_bw(0.25, 12)])]
    dom_y = fft_freqs[np.argmax(fft_y[freqs_bw(0.25, 12)])]
    dom_z = fft_freqs[np.argmax(fft_z[freqs_bw(0.25, 12)])]
    dom_m = fft_freqs[np.argmax(fft_m[freqs_bw(0.25, 12)])]
    
    return (np.array([mn_p, mn_x, mn_y, mn_z, dom_x, dom_y, dom_z, dom_m]), 
            ppg, accx, accy, accz)

def RunPulseRateAlgorithm(data_fl, ref_fl, fs = 125, window_length_s = 8, window_shift_s = 2):
    """ 
    Compute mean absolute error (MAE) and confidence from predictions.
    Args:
      data_fl: signal data
      ref_fl: reference data
      fs: sampling frequency (Hz)
      window_length_s: window size, signal length in seconds
      window_shift_s: signal shift in seconds
    Returns:
        numpy arrays for MAE between prediction-ground truth, and confidence about predictions.
    """
    # Load data using LoadTroikaDataFile   
    labels, features, signals = LoadTroikaDataFile(data_fl, ref_fl, fs, window_length_s, window_shift_s)
    error, confidence = [], []
    model = load_model()
    
    # Compute pulse rate estimates and estimation confidence.
    for i,feature in enumerate(features):
        pred = model.predict(np.reshape(feature, (1, -1)))[0]
        sum = np.sum(feature)
        sum_f = feature[0]*1.20 + feature[5]*1.1 + feature[7]*1.85
        conf = abs(sum_f/sum)
        error.append(np.abs((pred-labels[i])))
        confidence.append(conf)
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return np.array(error), np.array(confidence)

def train(fs=125, window_length_s=8, window_shift_s=2):
    """ 
    Model training using Adaboost Regressor.
    Args:
      fs: sampling frequency (Hz)
      window_length_s: window size, signal length in seconds
      window_shift_s: signal shift in seconds
    Returns:
        trained model
    """
    data_fls, ref_fls = LoadTroikaDataset()
    labels, features, signals = [], [], []
    
    for data_fl, ref_fl in (zip(data_fls, ref_fls)):
        label, feature, signal = LoadTroikaDataFile(data_fl, ref_fl, fs, window_length_s, window_shift_s)
        labels.extend(label)
        features.extend(feature)
        signals.extend(signal)
                            
    labels = np.array(labels)
    features = np.array(features)
    
    model = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10), learning_rate=0.001, n_estimators=300, random_state=42)
    kFolds = KFold(n_splits=8)
    splits = kFolds.split(features,labels)
    for i, (train_idx, test_idx) in enumerate(splits):
        X_train, y_train = features[train_idx], labels[train_idx]
        X_test, y_test = features[test_idx], labels[test_idx]
        model.fit(X_train, y_train)        
    pkl_filename = 'model.pkl'    
    with open(pkl_filename, 'wb') as file:
        pickle.dump(model, file) #saves the model in pickle format
    
    return model

def load_model():
    """
    It loads the model.
    """
    
    pkl_filename = 'model.pkl'
    pickle_model = None
    if (not os.path.isfile(pkl_filename)):
        train()
    
    with open(pkl_filename, 'rb') as file:
        pickle_model = pickle.load(file)
    return pickle_model

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a description of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

**Code Description** - In the previous code, we have developed a pulse rate algorithm that estimates pulse rate from a PPG signal (it optically measures blood flow at the wrist) and a IMU signal (3-axis accelerometer) embedded inside the device to measure motion (linear acceleration). 

To run the code and use the algorithm, we must first read the data from the Troika dataset with the LoadTroikaDataset() and LoadTroikaDataFile(). The next step is the preprocessing of the signals, which is performed by the bandpass_filter() and Featurize(). RunPulseRateAlgorithm() and train() are the main part of the code, where we develop and train our model. Finally, load_model(), AggregateErrorMetric(), and Evaluate() will trigger the execution of the algorithm and evaluate it's effectiveness.

**Data Description** - The data that was used to train and test the algorithm comes from the Troika dataset [1]. Troika includes experimental results on datasets recorded from 12 subjects during fast running at the peak speed of 15 km/hour. This dataset includes two data files for each subject: the data file and the reference file. These files are read in the algorithm by the LoadTroikaDataset() and LoadTroikaDataFile() respectively. The data comes from two different sensors (PPG and accelerometer) recording at 8Hz or 125ms time frames, and in total it contains 4 arrays of data (1 for the PPG and 3 for the accelerometer - x, y, and z axis). 

One major limitation of this dataset is the lack of demographic data. Due to this, it is not possible to make conclusions regarding gender and age. This is important since the pulse rate varies significantly regarding gender and age. With large enough datasets and more informative clinical metadata (for example, age and gender), wearables can be used to discover previously unknown phenomena that may advance clinical science.

[1] Zhilin Zhang, Zhouyue Pi, and Benyuan Liu. TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise. IEEE Trans. on Biomedical Engineering. 2015. DOI: 10.1109/TBME.2014.2359372.

**Algorithm Description** - The algorithm works on data form a PPG sensor and an accelerometer. PPG is an optical sensor with LEDs (shine a typically green light into the skin) and a photodetector (measures the reflected light). During the systole phase, when the ventricles contract and pump blood through the arteries, there are more blood cells that absorb the green light so there is less reflected light. On the contrary, during the diastole phase, when the heart relaxes and fills with blood, there are less blood cells and the photodetector sees an increase in reflected light. This oscillating waveform can be used to detect pulse rate.
However, during walking or running, we see another periodic signal in the PPG due to arm motion. We use the accelerometer signal of our wearable device to help us keep track of which periodic signal is caused by motion. Because the accelerometer is only sensing arm motion, any periodic signal in the accelerometer is likely not due to the heart beating, and only due to the arm motion. If our pulse rate estimator is picking a frequency that's strong in the accelerometer, it may be making a mistake.

The algorithm works following the next steps:
1. Loading data: LoadTroikaDataset(), LoadTroikaDataFile()
 - Load the data and return numpy arrays for PPG and accelerometer signals.
2. Preprocessing and build features: bandpass_filter(), Featurize()
 - We filter the signals using a bandpass filter that suppresses the frequencies outside a specific band. We use a passband (band of frequencies that we want to preserve) between 40 and 240 BPM (beats per minute) because that is the usual frequency range of the pulse rate. 
 - Calculate the magnitude of the acceleration.
 - Use Fourier Transform to transfer the magnitude to the frequency domain.
 - Pick the maximum acceleration in the frame as a main feature to the model.
3. Build a model: RunPulseRateAlgorithm(), train()
 - RunPulseRateAlgorithm() uses a window length of 8 seconds and produces an output at least every 2 seconds. 
 - RunPulseRateAlgorithm() returns the per-estimate mean absolute error (MAE) and an estimation confidence. In pulse rate estimation, having a confidence value can be useful if a user wants just a handful of high-quality pulse rate estimate per night. They can use the confidence algorithm to select the 20 most confident estimates at night and ignore the rest of the outputs.  A higher confidence value means that this estimate should be more accurate than an estimate with a lower confidence value. If our algorithm is picking a strong frequency component that's not present in the accelerometer, we can be relatively confident in the estimate. 
 - In train() we use a Decision Tree Regression with AdaBoost and K-Fold Cross Validation to train our model. It returns the model in pickle format.
4. Evaluate the model: AggregateErrorMetric(), Evaluate()
 - Evaluate() uses MAE and confidence returned from RunPulseRateAlgorithm() and runs the AggregateErrorMetric(), which computes the MAE at 90% availability.

**Algorithm Performance** - Performance was computed by using cross-validation, specifically K-fold cross-validation. In K-fold cross-validation, the training set is randomly split into K (usually between 5 to 10) subsets known as folds. Where K-1 folds are used to train the model and the other fold is used to test the model. This technique improves the high variance problem in a dataset as we are randomly selecting the training and test folds. 

The most relevant metric for users of the algorithm is the mean absolute error (MAE), which measures the average of the absolute difference between each ground truth and the predictions. The result of MAE must be less than 15 BPM for the algorithm's performance to pass. 

This algorithm would need further signal data (it only has data from 12 subjects), demographics data (to distinguish between genders and ages), analysis of errors and confidence evaluation to be generalizable on different datasets. The algorithm design would depend on the application, since how much error is tolerable depends on that. For example, if we were using these pulse rate estimates to compute long term trends over months, then we may be more robust to higher error variance. However, if we wanted to give information back to the user about a specific workout or night of sleep, we would require a much lower error. Therefore, although this algorithm is not generalizable to other datasets, it could be used as a starting point for building a general one.

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 